In [31]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import StandardScaler

In [32]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

scaler = StandardScaler()

train_images = train_images.reshape(train_images.shape[0], -1)
test_images = test_images.reshape(test_images.shape[0], -1)

train_images = scaler.fit_transform(train_images)
test_images = scaler.transform(test_images)

# train_images = train_images.reshape(train_images.shape[0], 28, 28)
# test_images = test_images.reshape(test_images.shape[0], 28, 28)

train_labels = keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = keras.utils.to_categorical(test_labels, num_classes=10)

train_images = train_images / 255.0
test_images = test_images / 255.0

train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [33]:
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(784,)),
    keras.layers.Dense(10, activation="softmax")
])

def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result()) for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics, end=end)


n_epochs = 5
batch_size = 32
n_steps = len(train_images) // batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.categorical_crossentropy
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]


for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(train_images, train_labels)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
        loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(train_labels), mean_loss, metrics)
    print_status_bar(len(train_labels), len(train_labels), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_state()


c:\programming\ml\my_env\lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/5
48000/48000 - mean: 0.9037 - mean_absolute_error: 0.0996
48000/48000 - mean: 0.9037 - mean_absolute_error: 0.0996


AttributeError: 'Mean' object has no attribute 'reset_states'